In [29]:
from __future__ import print_function
import pandas as pd
import numpy as np
import xlsxwriter
import cv2
import matplotlib.colors as cs
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import urllib
from pyinaturalist import get_observations
import urllib.request
from matplotlib import pyplot as plt
import matplotlib

# Color Hex Codes to Color Names
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

from PIL import Image
from sklearn.cluster import MiniBatchKMeans

### Create a new structured dataframe using the user id

In [30]:
iNaturalist_Username = input()

In [31]:
# Create a new observation
observations = get_observations(user_id=iNaturalist_Username, page='all')

# df_list is used to append all the observations into a dataframe
df_list = []
# df_url is used to store image urls
df_url = []
# df_lat_long is used to store latitude and longitude
df_lat = []
df_long = []
# df_site_url is used to store site url
df_site_url = []
# Store user id and login
# df_user_id = []
df_user_login = []
# Store public/positional accuracy
# df_pos_acc = []
# df_pub_pos_acc = []
# Store annotations
# df_annotations = []
# Store place guess
df_place = []
#Store time_observed_at
# df_time = []

for obs in observations['results']:
    # From dict to dataframe
    df = pd.DataFrame.from_dict(obs, orient='index')
    # Get image urls
    if obs.get('photos') is not None:
        image_url = obs.get('photos')[0].get('url')
    else:
        image_url = 'None'
    # Get latitude and longitude
    # Get positional accuracy
    if obs.get('location') is not None:
        lat = obs.get('location')[0]
        lon = obs.get('location')[1]
        # pos_acc = obs.get('positional_accuracy')
        # pub_pos_acc = obs.get('public_positional_accuracy')
    else:
        lat = '             40.343137'
        lon = '             74.655070'
    # Get annotations
    # if obs.get('annotations') is not None:
    #     anno = obs.get('annotations')
    # else:
    #     anno = 'Princeton University'
    # anno = obs.get('annotations')
        # pos_acc = 'None'
        # pub_pos_acc = 'None'
    
    # Get place guess
    if obs.get('place_guess') is not None and lat != '             40.343137' and lon != '             74.655070':
        place_guess = obs.get('place_guess')
    else:
        place_guess = 'Princeton University'    
    
    # Get site url
    if obs.get('uri') is not None:
        site_url = obs.get('uri')
    else:
        site_url = 'None'
    # Get user id and login
    if obs.get('user') is not None:
        #user_id = obs.get('user').get('id')
        user_login = obs.get('user').get('login')
    else:
        #user_id = 'None'
        user_login = 'None'
    # if obs.get('time_observed_at') is not None:
    #     time_at = obs.get('time_observed_at')
    # else:
    #     time_at = 'None'
        
    # Transpose the dataframe so that rows represent images and columns are variables
    # that describe the images.
    df = df.transpose()
    df_list.append(df)
    df_url.append(image_url)
    df_lat.append(lat)
    df_long.append(lon)
    df_site_url.append(site_url)
    #df_user_id.append(user_id)
    df_user_login.append(user_login)
    # df_pos_acc.append(pos_acc)
    # df_pub_pos_acc.append(pub_pos_acc)
    # df_annotations.append(anno)
    df_place.append(place_guess)
    # df_time.append(time_at)

# Concatenate all dataframes
iNatStruct = pd.concat(df_list)

# Add updated columns to the dataframe
iNatStruct['square_image_url'] = df_url
iNatStruct['latitude'] = df_lat
iNatStruct['longitude'] = df_long
iNatStruct['site_url'] = df_site_url
#iNatStruct['user_id'] = df_user_id
iNatStruct['User'] = df_user_login
# iNatStruct['positional_accuracy'] = df_pos_acc
# iNatStruct['public_positional_accuracy'] = df_pub_pos_acc
# iNatStruct['annotations'] = df_annotations
iNatStruct['place_guess'] = df_place
# iNatStruct['time_observed_at'] = df_time

# Replace image size from 'square' to 'medium' and 'large'
iNatStruct['Image_Link'] = iNatStruct['square_image_url'].replace('square', 'medium', regex=True)
# iNatStruct['large_image_url'] = iNatStruct['square_image_url'].replace('square', 'large', regex=True)

# Reset index and drop the old index
iNatStruct = iNatStruct.reset_index()
# iNatStruct = iNatStruct.drop(['index'], axis=1)

iNatStruct.head(3)
# observations['results']

,index,quality_grade,time_observed_at,taxon_geoprivacy,annotations,uuid,id,cached_votes_total,identifications_most_agree,species_guess,...,observation_photos,faves,non_owner_ids,observed_on,square_image_url,latitude,longitude,site_url,User,Image_Link
0,0,needs_id,2022-05-25T12:07:21-04:00,open,[],7db04325-38f7-4029-9dc9-766616b94437,118661315,0,False,None,...,"[{'id': 190373088, 'position': 0, 'uuid': 'd0d...",[],"[{'hidden': False, 'disagreement': None, 'flag...",2022-05-25 12:07:21-05:00,https://inaturalist-open-data.s3.amazonaws.com...,37.246322,-80.409523,https://www.inaturalist.org/observations/11866...,llhouse2,https://inaturalist-open-data.s3.amazonaws.com...
1,0,research,2022-05-25T12:09:03-04:00,obscured,[],7fe69598-d795-4241-9cec-c2e5c5a61151,118661513,0,True,Carolina sweetshrub,...,"[{'id': 190372817, 'position': 0, 'uuid': 'c70...",[],"[{'hidden': False, 'disagreement': None, 'flag...",2022-05-25 12:09:03-05:00,https://inaturalist-open-data.s3.amazonaws.com...,37.350497,-80.484791,https://www.inaturalist.org/observations/11866...,llhouse2,https://inaturalist-open-data.s3.amazonaws.com...
2,0,needs_id,2022-05-25T12:10:21-04:00,None,[],440685b3-675a-4eb7-8817-4f5964eb53f0,118661745,0,False,None,...,"[{'id': 190372696, 'position': 0, 'uuid': '6ba...",[],[],2022-05-25 12:10:21-05:00,https://inaturalist-open-data.s3.amazonaws.com...,37.246163,-80.409251,https://www.inaturalist.org/observations/11866...,llhouse2,https://inaturalist-open-data.s3.amazonaws.com...


## Continue on initial structured data frame 

In [32]:
# Split time_observed_at into date and time
def convertT(time):
    time = time.replace('T',' ')
    return time

iNatStruct.time_observed_at.map(lambda t: convertT(t).split())[0][1]

'12:07:21-04:00'

In [33]:
# Use lambda function to split time_observed_at into date and time
for i in iNatStruct.time_observed_at:    
    iNatStruct['observed_date'] = (iNatStruct.time_observed_at.map(lambda t: convertT(t).split()[0]))
    iNatStruct['observed_time'] = (iNatStruct.time_observed_at.map(lambda t: convertT(t).split()[1]))

In [34]:
# shift columns observed_date and observed_time to position 1 and 2
column1 = iNatStruct.pop('observed_date')
column2 = iNatStruct.pop('observed_time')
  
# insert column using insert
iNatStruct.insert(1, 'observed_date', column1)
iNatStruct.insert(2, 'observed_time', column2)

# delete time_observed_at column
del iNatStruct["time_observed_at"]
del iNatStruct["observed_on"]
#del iNatStruct["observed_time"]

In [35]:
# shift location column to last column
column1 = iNatStruct.pop('location')
  
# insert location column using insert
iNatStruct.insert(len(iNatStruct) - 1, 'location', column1) 
# not sure why it's not moving the column to last position

In [36]:
# Remove the characters after '-'
def removeChar(char):
    ch = "-"
    char = char.split(ch, 1)[0]
    return char

# Use lambda function to remove the characters after '-'
iNatStruct['observed_time'] = iNatStruct.observed_time.map(lambda t: removeChar(t))

In [37]:
# Remove the characters after '+'
def removeChara(char):
    ch = "+"
    char = char.split(ch, 1)[0]
    return char

# Use lambda function to remove the characters after '+'
iNatStruct['observed_time'] = iNatStruct.observed_time.map(lambda t: removeChara(t))

In [38]:
# Time to Second
def timeToSec(time):
    time = time.split(':')
    sec = int(time[0])*3600 + int(time[1])*60 + int(time[2])
    return sec

# Use lambda function to convert time to second
iNatStruct['Seconds'] = iNatStruct.observed_time.map(lambda t: timeToSec(t))

In [39]:
# Rename columns
iNatStruct.rename(columns = {'species_guess':'Species', 'observed_date':'Date', 'observed_time':'Time', 
                             'place_guess':'Place', 'latitude':'Lat', 'longitude':'Long'}, inplace = True)

# The structured dataframe is now ready
iNatStruct = iNatStruct[["Image_Link", "Species", "User", "Date", "Time", "Seconds", "Place", "Lat", "Long"]]

# Sort DataFrame by date and time
iNatStruct = iNatStruct.sort_values(['Date', 'Time'], ascending = [True, True])

# Add p_ column
p_lab = ['p'+str(s+1) for s in iNatStruct.index]
iNatStruct.insert(0, "Image_Label", p_lab)

# Sort DataFrame by Image_Label
iNatStruct = iNatStruct.sort_values('Image_Label', key=lambda x: x.str.extract('(\d+)').squeeze().astype(int))

iNatStruct

,Image_Label,Image_Link,Species,User,Date,Time,Seconds,Place,Lat,Long
0,p1,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-05-25,12:07:21,43641,"Highview Dr, Blacksburg, VA, US",37.246322,-80.409523
1,p2,https://inaturalist-open-data.s3.amazonaws.com...,Carolina sweetshrub,llhouse2,2022-05-25,12:09:03,43743,"Virginia, US",37.350497,-80.484791
2,p3,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-05-25,12:10:21,43821,"Highview Dr, Blacksburg, VA, US",37.246163,-80.409251
3,p4,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-05-25,12:10:37,43837,"Highview Dr, Blacksburg, VA, US",37.246379,-80.409562
4,p5,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-06-03,10:48:19,38899,Princeton University,40.343137,74.655070
5,p6,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-06-03,10:41:30,38490,Princeton University,40.343137,74.655070
6,p7,https://inaturalist-open-data.s3.amazonaws.com...,Moosewood,llhouse2,2022-06-09,10:10:02,36602,"Jefferson National Forest, Virginia 24134, USA",37.250402,-80.666454
7,p8,https://inaturalist-open-data.s3.amazonaws.com...,American Chestnut,llhouse2,2022-06-09,10:10:00,36600,"Giles County, VA, USA",37.249602,-80.666705
8,p9,https://inaturalist-open-data.s3.amazonaws.com...,American chestnut,llhouse2,2022-06-09,10:10:00,36600,"Giles County, VA, USA",37.248573,-80.667484
9,p10,https://inaturalist-open-data.s3.amazonaws.com...,black oak,llhouse2,2022-06-09,10:10:00,36600,"Jefferson National Forest, Virginia 24134, USA",37.250402,-80.666454


# Color Extraction

#### Define centroid_histogram Function 
- shows percentage of dominant (most frequent) color within image using kmeans

In [40]:
def centroid_histogram(clt):
    # grab the number of different clusters and create a histogram
    # based on the number of pixels assigned to each cluster
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # normalize the histogram, so that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()
    #hist = "{:.2f}".format(hist)
    
    # return the histogram
    return hist

#### Define Color Palette Display Function

In [41]:
def palette(clusters):
    width = 300
    palette = np.zeros((50, width, 3), np.uint8)
    steps = width/clusters.cluster_centers_.shape[0]
    for idx, centers in enumerate(clusters.cluster_centers_):
        palette[:, int(idx*steps):(int((idx+1)*steps)), :] = centers
    return palette

### Implement RGB Color Columns

In [42]:
# column name list 
col_names =  ['R1', 'G1', 'B1', 'R2', 'G2', 'B2', 'R3', 'G3', 'B3']
col_names1 =  ['Set1(R,G,B)', 'Set2(R,G,B)', 'Set3(R,G,B)']
col_names2 =  ['Set1(R,G,B,Prop)', 'Set2(R,G,B,Prop)', 'Set3(R,G,B,Prop)']
  
# create an empty dataframe with columns
RGBdf = pd.DataFrame(columns = col_names)
hexDF = pd.DataFrame(columns = col_names1)
colorDF = pd.DataFrame(columns = col_names2) 

### Implement Color Names Column

In [43]:
# Convert RGB to Color Names
def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return f'{names[index]}'

# Testing
# print(convert_rgb_to_names((191.3446284379988,206.0364699381563,221.936275645711)))

### Color Extraction for Requested Images

In [44]:
# n_init default:10
# max_iter default:300

hex_code = []
ord_hex_code = []
r_pix = []
g_pix = []
b_pix = []
per = []

cluster_count = 3
#clt = KMeans(n_clusters=cluster_count, n_init = 3, max_iter = 150) 
clt = MiniBatchKMeans(n_clusters = cluster_count)

for img in iNatStruct["Image_Link"]: 
    req = urllib.request.urlopen(img)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    imgNat = cv2.imdecode(arr, -1)
    imgray = cv2.cvtColor(imgNat, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(imgray, (5,5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    imgNat = cv2.cvtColor(imgNat,cv2.COLOR_BGR2RGB) #convert bgr to rgb
    # grab largest contour
    largest_contour = max(contours, key=cv2.contourArea)

    # create a black `mask` the same size as the original grayscale image 
    mask = np.zeros_like(imgray)
    # fill new mask with the largest contour shape
    cv2.fillPoly(mask, [largest_contour], 255)
    # create a copy of current mask
    res_mask = np.copy(mask)
    res_mask[mask == 0] = cv2.GC_BGD # obvious background pixels
    res_mask[mask == 255] = cv2.GC_PR_BGD # probable background pixels
    res_mask[mask == 255] = cv2.GC_FGD # obvious foreground pixels

    # create a mask for obvious and probable foreground pixels
    mask2 = np.where((res_mask == cv2.GC_FGD) | (res_mask == cv2.GC_PR_FGD),255,0).astype('uint8')

    # create 'new_mask3d' from 'mask2' but with 3 dimensions instead 2
    new_mask3d = np.repeat(mask2[:, :, np.newaxis], 3, axis=2)
    mask3d = new_mask3d
    mask3d[new_mask3d > 0] = 255.0
    mask3d[mask3d > 255] = 255.0
    
    # apply Gaussian blurring to smoothen out the edges
    mask3d = cv2.GaussianBlur(mask3d, (5, 5), 0)

    # create the foreground image by zeroing out the pixels where `mask2` has black pixels
    foreground = np.copy(imgNat).astype(float)
    foreground[mask2 == 0] = 0
    
    # save new image with black background in variable 'imgfile'
    imgfile = Image.fromarray(foreground.astype(np.uint8))
    # only pass through non-transparent pixels, i.e. those where A!=0 in the RGBA quad
    na = np.array([f for f in imgfile.getdata() if f[2] !=0], np.uint8)
    
    # apply clt.fit to na
    clt.fit(na)
    
    npbins = np.arange(0, cluster_count + 1)
    hist = np.histogram(clt.labels_, bins=npbins)
    labels = np.unique(clt.labels_)
    colors_w_black_back = []
    colors = [] # list to hold color
    hexlabels = [] # list to hold hex color code
    c = []
    
    # add percentage
    histPer = list(centroid_histogram(clt))
    per.append([round(percent,2) for percent in histPer])
    
    # extract 3 main color
    for i in range(clt.cluster_centers_.shape[0]):
        # store cluster center to clust_cent
        clust_cent = clt.cluster_centers_[i]
        
        # order color using proportion
        colors.append(tuple(clust_cent) + (histPer[i], ))
        colors = sorted(colors, key = lambda element : element[3], reverse=True)
        
        # order hex color code using proportion
        c.append(tuple(clust_cent/255) + (histPer[i], ))
        c = sorted(c, key = lambda element : element[3], reverse=True)
    
    # remove proportion from list
    output = list(map(lambda elem: elem[0:3], c))

    # append color
    colorDF = colorDF.append(pd.DataFrame([colors], columns=col_names2), 
                   ignore_index = True)
    # append ordered hex code
    ord_hex_code.append([matplotlib.colors.to_hex( output[i]) for i in range(cluster_count)])

C:\Users\18334\AppData\Local\Temp\ipykernel_4236\3919908432.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  colorDF = colorDF.append(pd.DataFrame([colors], columns=col_names2),
C:\Users\18334\AppData\Local\Temp\ipykernel_4236\3919908432.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  colorDF = colorDF.append(pd.DataFrame([colors], columns=col_names2),
C:\Users\18334\AppData\Local\Temp\ipykernel_4236\3919908432.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  colorDF = colorDF.append(pd.DataFrame([colors], columns=col_names2),
C:\Users\18334\AppData\Local\Temp\ipykernel_4236\3919908432.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

### Add R, G, B, Prop Features to iNatStruct

In [45]:
col_1 =  ['R1', 'G1', 'B1', 'Prop1']
col_2 =  ['R2', 'G2', 'B2', 'Prop2']
col_3 =  ['R3', 'G3', 'B3', 'Prop3']

In [46]:
combinedColorDF = pd.DataFrame()
combinedColorDF[col_1] = pd.DataFrame(colorDF['Set1(R,G,B,Prop)'].tolist(), index=colorDF.index)
combinedColorDF[col_2] = pd.DataFrame(colorDF['Set2(R,G,B,Prop)'].tolist(), index=colorDF.index)
combinedColorDF[col_3] = pd.DataFrame(colorDF['Set3(R,G,B,Prop)'].tolist(), index=colorDF.index)

In [47]:
# Store RGB Color Names
Color_Names_1 = []
Color_Names_2 = []
Color_Names_3 = []

for i in combinedColorDF.index:
    # RGB1
    rgb1 = combinedColorDF.iloc[i, 0:3]
    Color_Names_1.append(convert_rgb_to_names(rgb1))
    # RGB2
    rgb2 = combinedColorDF.iloc[i, 4:7]
    Color_Names_2.append(convert_rgb_to_names(rgb2))
    # RGB3
    rgb3 = combinedColorDF.iloc[i, 8:11]
    Color_Names_3.append(convert_rgb_to_names(rgb3))

In [48]:
# re-set index to be same as iNatStruct
combinedColorDF = combinedColorDF.set_index(iNatStruct.index)

In [49]:
# append method
result = pd.concat([iNatStruct, combinedColorDF], axis=1)

In [50]:
iNatStruct = result
iNatStruct["Hex_Color_Code"] = ord_hex_code

In [51]:
# Convert 2D list to numpy.ndarray and get the transposed object with the T attribute
# Remove and then add RGB columns to different location
iNatStruct['RGB1'] = pd.DataFrame(np.array([Color_Names_1]).T)
RGB1 = iNatStruct.pop('RGB1')
iNatStruct.insert(13, 'RGB1', RGB1)

iNatStruct['RGB2'] = pd.DataFrame(np.array([Color_Names_2]).T)
RGB2 = iNatStruct.pop('RGB2')
iNatStruct.insert(18, 'RGB2', RGB2)

iNatStruct['RGB3'] = pd.DataFrame(np.array([Color_Names_3]).T)
RGB3 = iNatStruct.pop('RGB3')
iNatStruct.insert(23, 'RGB3', RGB3)

Hex_Code = iNatStruct.pop('Hex_Color_Code')
iNatStruct.insert(10, 'Hex_Color_Code', Hex_Code)

# Size Feature Extraction

#### Define tag function: detects edges of object, tags with a rectangle and returns the width and height

In [52]:
def tag(img):
    blurred = cv2.blur(img, (3,3))
    canny = cv2.Canny(blurred, 50, 200)
    
    pts = np.argwhere(canny>0)
    y1,x1 = pts.min(axis=0)
    y2,x2 = pts.max(axis=0)
    
    w = (x2-x1) # width
    h = (y2-y1) # height
    
    return [x1, y1, w,h]

In [53]:
contArea = []
width = []
height = []
ratio = []
extentInfo = []
solidityInfo = []
angleInfo = []

for img in iNatStruct["Image_Link"]: 
    req = urllib.request.urlopen(img)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    imgNat = cv2.imdecode(arr, -1)
    imgray = cv2.cvtColor(imgNat, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(imgray, (5,5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    imgNat = cv2.cvtColor(imgNat,cv2.COLOR_BGR2RGB) 
    largest_contour = max(contours, key=cv2.contourArea) # maximum contour
    
    cv2.drawContours(imgNat, largest_contour, -1, (0, 255, 0), 3)
    
    # Contour Area
    #cnt = contours[0] # smallest contour
    cnt = largest_contour
    
    # tagged attributes
    x_tag, y_tag, w_tag, h_tag = tag(imgNat)
    
    # set bounding rectangle around object
    x1,y1,w1,h1 = cv2.boundingRect(cnt)
    
    # Contour Area
    area = cv2.contourArea(cnt)
    
    # Orientation (Angle)
    if area < 5:
        angle = float("NaN")
    else:
        (x,y),(MA,ma),angle = cv2.fitEllipse(cnt)
    
    # Aspect Ratio
    aspect_ratio = float(w_tag)/(h_tag)
    
    # Extent
    rect_area = w_tag*h_tag
    extent = float(area)/rect_area

    # Solidity
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull)
    
    # replace solidity to Nan when hull_area is 0
    if hull_area == 0:
        solidity = float("NaN") 
        
    else:
        solidity = float(area)/hull_area
        
    contArea.append(area)
    width.append(w_tag)
    height.append(h_tag)
    ratio.append(aspect_ratio)
    extentInfo.append(extent)
    solidityInfo.append(solidity)
    angleInfo.append(angle)

In [54]:
iNatStruct["Width"] = width
iNatStruct["Height"] = height
iNatStruct["Contour_Area"] = contArea
iNatStruct["Aspect_Ratio"] = ratio
iNatStruct["Extent"] = extentInfo
iNatStruct["Solidity"] = solidityInfo
iNatStruct["Angle"] = angleInfo
iNatStruct.head()

,Image_Label,Image_Link,Species,User,Date,Time,Seconds,Place,Lat,Long,...,B3,RGB3,Prop3,Width,Height,Contour_Area,Aspect_Ratio,Extent,Solidity,Angle
0,p1,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-05-25,12:07:21,43641,"Highview Dr, Blacksburg, VA, US",37.246322,-80.409523,...,57.729569,darkolivegreen,0.045747,254,242,42647.0,1.049587,0.693808,0.919077,105.935860
1,p2,https://inaturalist-open-data.s3.amazonaws.com...,Carolina sweetshrub,llhouse2,2022-05-25,12:09:03,43743,"Virginia, US",37.350497,-80.484791,...,22.023270,black,0.280623,374,345,60445.0,1.084058,0.468457,0.656854,46.177895
2,p3,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-05-25,12:10:21,43821,"Highview Dr, Blacksburg, VA, US",37.246163,-80.409251,...,109.283702,dimgray,0.019015,295,119,21082.0,2.478992,0.600541,0.898311,89.916687
3,p4,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-05-25,12:10:37,43837,"Highview Dr, Blacksburg, VA, US",37.246379,-80.409562,...,120.671477,gray,0.029520,267,152,19528.5,1.756579,0.481187,0.741387,83.742508
4,p5,https://inaturalist-open-data.s3.amazonaws.com...,None,llhouse2,2022-06-03,10:48:19,38899,Princeton University,40.343137,74.655070,...,33.018470,darkslategray,0.073504,499,458,126945.5,1.089520,0.555458,0.774320,63.340115


In [55]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('iNatStruct.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
iNatStruct.to_excel(writer, sheet_name='Sheet1', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [56]:
# convert to csv file
iNatStruct.to_csv('iNatStruct.csv', index=False)